# Google Form -> EDSL
This notebook provides example EDSL code for converting a non-EDSL survey into in an EDSL survey. This can be useful for accessing EDSL's built-in methods for analyzing survey data, and extending it with responses simulated with AI agents and diverse large language models.

[EDSL is an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Designing the task as an EDSL survey
We design the task as an EDSL survey *about* the survey to be converted: a series of questions prompting a language model to read and reformat the contents of a given survey. The formatted responses of the language model are readily usable components of a new EDSL survey that can be administered to AI agents and/or human audiences.

## Creating a meta-survey
We start by selecting appropriate question types for reformatting the contents of a given survey.
[EDSL comes with many common question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from a model: multiple choice, checkbox, free text, linear scale, etc.

Here we use `QuestionList` to return information about all the questions in the survey at once, as a list. We create a sequence of questions, using the response to one question as an input to the next question. This step-wise approach can improve performance by allowing a model to focus on distinct tasks, and also allow us to pinpoint modifications to instructions as needed (some models will perform better and need fewer instructions than others). Note that we use a `{{ placeholder }}` for the text of the survey to be reformatted that we want to add to the initial question, which allows us to reuse it with other content (e.g., another survey):

In [1]:
from edsl import QuestionList

First we ask the model to return just the questions from the survey:

In [2]:
q1 = QuestionList(
    question_name="q_text",
    question_text="""
    You are being asked to extract questions from the text of a survey.
    Read the text and then return a list of all the questions in the 
    order that you find them. Return only the list of questions.
    Survey: {{ scenario.text }}
    """
)

Next we ask the model to format the questions as dictionaries, and specify the question text and type:

In [3]:
q2 = QuestionList(
    question_name="q_type",
    question_text="""
    Now create a dictionary for each question, using keys 'question_text' and 'question_type'.
    The value for 'question_text' is the question text you already identified.
    The value for 'question_type' should be the most appropriate of the following types:
    'multiple_choice', 'checkbox', 'linear_scale' or 'free_text'.
    Return only the list of dictionaries you have created, with the 2 key/value pairs for each question.
    """
)

Next we ask the model to add the question options (if any):

In [4]:
q3 = QuestionList(
    question_name="q_options",
    question_text="""
    Now add a key 'question_options' to each dictionary for all questions that are not free text,
    with a value that is a list of the answer options for the question.
    Preserve any integer options as integers, not strings.
    If there are labels for linear scale answer options then add another key 'option_labels'
    with a value that is a dictionary: the keys are the relevant integers and the values are the labels.
    Return only the list of dictionaries you have created with all relevant key/value pairs for each question.
    """
)

Finally, we ask the model to give each question a name:

In [5]:
q4 = QuestionList(
    question_name="q_name",
    question_text="""
    Now add a key 'question_name' to each dictionary.
    The value should be a unique short pythonic string.
    Return only the list of dictionaries that you have created, 
    with all the key/value pairs for each question.
    """
)

Next we combine the questions into a `Survey` in order to administer them together. 
We add a "memory" of each prior question in the survey so that the model will have the context and its answers on hand when answering each successive question:

In [6]:
from edsl import Survey

In [7]:
survey = Survey(questions = [q1, q2, q3, q4]).set_full_memory_mode()

## Adding content to questions
Next we create a `Scenario` object for the contents of a (non-EDSL) survey to be inserted in the first question. 
This allows us to reuse the questions with other content. [Learn more about using scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html) to scale data labeling and other tasks.

Here we create a scenario for a [Google Form](https://forms.gle/GufjVVs5PfxUeyoj7) (a customer feedback survey) that we have stored as a publicly-accessible PDF at [Coop](https://www.expectedparrot.com/content/explore).

Code for posting a PDF to the Coop (rerun with your own file):

In [1]:
from edsl import FileStore

if refresh := False:
    fs = FileStore("customer_feedback_survey.pdf") # file type is automatically inferred
    info = fs.push(
    description = "Example customer feedback survey",
    alias = "mock-survey",
    visibility = "public"
    )
    info


info = {'description': 'Example customer feedback survey',
 'object_type': 'scenario',
 'url': 'https://www.expectedparrot.com/content/d8bdddd2-7af9-49b1-86b7-51ebed2f71e1',
 'uuid': 'd8bdddd2-7af9-49b1-86b7-51ebed2f71e1',
 'version': '0.1.47.dev1',
 'visibility': 'public'}

Retrieving a file (replace with UUID of any desired object at Coop):

In [2]:
pdf_file = FileStore.pull(info["uuid"])

Creating a scenario for the content:

In [10]:
from edsl import Scenario

In [11]:
s = Scenario({"text":pdf_file.to_tempfile()})
s

,key,value
0,text,/var/folders/j0/xq1nxxt51j7_1dgv8s116fmh0000gn/T/tmp5cqcqowt.pdf


## Selecting language models
EDSL works with many popular language models that we can select to use in generating survey responses. 
You can provide your own API keys for models or activate remote inference to run surveys at the Expected Parrot server with any available models. 
[Learn more about working with language models](https://docs.expectedparrot.com/en/latest/language_models.html) and using [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html).

In [12]:
from edsl import ModelList, Model

A list of all available models can be viewed at the [model pricing and performance page](https://www.expectedparrot.com/getting-started/coop-pricing).

In [13]:
# Model.available()

Here we select several models to compare their responses:

In [14]:
models = ModelList(
    Model(m) for m in ["gemini-1.5-flash", "gpt-4o", "claude-3-5-sonnet-20240620"]
)

## Running a survey
Next we add the scenario and models to the survey and run it. 
This generates a dataset of `Results` that we can access with built-in methods for analysis. 
[Learn more about working with results](https://docs.expectedparrot.com/en/latest/results.html).

In [15]:
results = survey.by(s).by(models).run()

Job UUID,3863fbb2-4bca-4480-be93-9972fbb13e48
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/3863fbb2-4bca-4480-be93-9972fbb13e48
Exceptions Report URL,None
Results UUID,0d31b57d-be32-471a-a0ec-96a072eee19b
Results URL,https://www.expectedparrot.com/content/0d31b57d-be32-471a-a0ec-96a072eee19b


To see a list of all the components of the results that have been generated:

In [16]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.q_name
4,answer.q_options
5,answer.q_text
6,answer.q_type
7,cache_keys.q_name_cache_key
8,cache_keys.q_options_cache_key
9,cache_keys.q_text_cache_key


We can filter, sort, select and print components in a table:

In [17]:
(
    results
    .sort_by("model")
    .select("model", "q_name") #"q_text", "q_type", "q_options", "q_name")
)

,model.model,answer.q_name
0,claude-3-5-sonnet-20240620,"[{'question_text': 'How many times have you visited this store in the past month?', 'question_type': 'multiple_choice', 'question_options': [0, 1, 2, 3, '4 or more'], 'question_name': 'visit_frequency'}, {'question_text': 'How satisfied were you with your overall shopping experience?', 'question_type': 'linear_scale', 'question_options': [1, 2, 3, 4, 5], 'option_labels': {'1': 'Very Dissatisfied', '5': 'Very Satisfied'}, 'question_name': 'overall_satisfaction'}, {'question_text': 'How likely are you to recommend this store to a friend or family member?', 'question_type': 'linear_scale', 'question_options': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'option_labels': {'0': 'Not at all likely', '10': 'Extremely likely'}, 'question_name': 'recommendation_likelihood'}, {'question_text': 'How would you rate the cleanliness of the store?', 'question_type': 'linear_scale', 'question_options': [1, 2, 3, 4, 5], 'question_name': 'store_cleanliness'}, {'question_text': 'How would you rate the friendliness of our staff?', 'question_type': 'linear_scale', 'question_options': [1, 2, 3, 4, 5], 'question_name': 'staff_friendliness'}, {'question_text': 'How would you rate the variety of products available?', 'question_type': 'linear_scale', 'question_options': [1, 2, 3, 4, 5], 'question_name': 'product_variety'}, {'question_text': 'How would you rate the prices of our products?', 'question_type': 'linear_scale', 'question_options': [1, 2, 3, 4, 5], 'question_name': 'product_prices'}, {'question_text': 'What is your age range?', 'question_type': 'multiple_choice', 'question_options': ['Under 18', '18-24', '25-34', '35-44', '45-54', '55-64', '65 or older'], 'question_name': 'age_range'}, {'question_text': 'What is your gender?', 'question_type': 'multiple_choice', 'question_options': ['Male', 'Female', 'Non-binary', 'Prefer not to say'], 'question_name': 'gender'}, {'question_text': 'Do you have any additional comments or suggestions for improvement?', 'question_type': 'free_text', 'question_name': 'additional_comments'}]"
1,gemini-1.5-flash,"[{'question_text': 'What is your age?', 'question_type': 'free_text', 'question_name': 'age'}, {'question_text': 'What is your gender?', 'question_type': 'multiple_choice', 'question_options': [], 'question_name': 'gender'}, {'question_text': 'What is your highest level of education?', 'question_type': 'multiple_choice', 'question_options': [], 'question_name': 'education'}, {'question_text': 'What is your annual household income?', 'question_type': 'free_text', 'question_name': 'income'}, {'question_text': 'How many people live in your household?', 'question_type': 'free_text', 'question_name': 'household_size'}, {'question_text': 'How many children live in your household?', 'question_type': 'free_text', 'question_name': 'children'}, {'question_text': 'What is your marital status?', 'question_type': 'multiple_choice', 'question_options': [], 'question_name': 'marital_status'}, {'question_text': 'What is your race/ethnicity?', 'question_type': 'multiple_choice', 'question_options': [], 'question_name': 'race'}, {'question_text': 'How would you rate your overall health?', 'question_type': 'linear_scale', 'question_options': [1, 2, 3, 4, 5], 'option_labels': {'1': 'Poor', '2': 'Fair', '3': 'Good', '4': 'Very Good', '5': 'Excellent'}, 'question_name': 'health'}, {'question_text': 'Do you have any chronic health conditions?', 'question_type': 'multiple_choice', 'question_options': [True, False], 'question_name': 'chronic_conditions'}, {'question_text': 'If yes, please specify.', 'question_type': 'free_text', 'question_name': 'chronic_conditions_details'}, {'question_text': 'Do you have health insurance?', 'question_type': 'multiple_choice', 'question_options': [True, False], 'question_name': 'health_insurance'}, {'question_text': 'If yes, what type of health insurance do you have?', 'question_type': 'multiple_choice', 'question_options': [], 'question_name': 'ins

## Creating a new EDSL survey
Now we can construct a new EDSL survey with the reformatted components of the original survey.
This is done by creating `Question` objects with the question components, passing them to a new `Survey`, and then optionally designing and assigning AI agents to answer the survey.

Here we select one of the model's responses to use:

In [18]:
from edsl import Question

In [19]:
questions_list = results.filter("model.model == 'claude-3-5-sonnet-20240620'").select("q_name").to_list()[0]
questions_list

[{'question_text': 'How many times have you visited this store in the past month?',
  'question_type': 'multiple_choice',
  'question_options': [0, 1, 2, 3, '4 or more'],
  'question_name': 'visit_frequency'},
 {'question_text': 'How satisfied were you with your overall shopping experience?',
  'question_type': 'linear_scale',
  'question_options': [1, 2, 3, 4, 5],
  'option_labels': {'1': 'Very Dissatisfied', '5': 'Very Satisfied'},
  'question_name': 'overall_satisfaction'},
 {'question_text': 'How likely are you to recommend this store to a friend or family member?',
  'question_type': 'linear_scale',
  'question_options': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'option_labels': {'0': 'Not at all likely', '10': 'Extremely likely'},
  'question_name': 'recommendation_likelihood'},
 {'question_text': 'How would you rate the cleanliness of the store?',
  'question_type': 'linear_scale',
  'question_options': [1, 2, 3, 4, 5],
  'question_name': 'store_cleanliness'},
 {'question_text': 'Ho

In [20]:
edsl_questions = [Question(**q) for q in questions_list]
edsl_questions

[Question('multiple_choice', question_name = """visit_frequency""", question_text = """How many times have you visited this store in the past month?""", question_options = [0, 1, 2, 3, '4 or more']),
 Question('linear_scale', question_name = """overall_satisfaction""", question_text = """How satisfied were you with your overall shopping experience?""", question_options = [1, 2, 3, 4, 5], option_labels = {1: 'Very Dissatisfied', 5: 'Very Satisfied'}),
 Question('linear_scale', question_name = """recommendation_likelihood""", question_text = """How likely are you to recommend this store to a friend or family member?""", question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], option_labels = {0: 'Not at all likely', 10: 'Extremely likely'}),
 Question('linear_scale', question_name = """store_cleanliness""", question_text = """How would you rate the cleanliness of the store?""", question_options = [1, 2, 3, 4, 5], option_labels = {}),
 Question('linear_scale', question_name = """staff_frien

In [21]:
new_survey = Survey(edsl_questions)

We can inspect the survey that has been created:

In [22]:
new_survey

,question_type,question_name,question_options,question_text,option_labels
0,multiple_choice,visit_frequency,"[0, 1, 2, 3, '4 or more']",How many times have you visited this store in the past month?,nan
1,linear_scale,overall_satisfaction,"[1, 2, 3, 4, 5]",How satisfied were you with your overall shopping experience?,"{1: 'Very Dissatisfied', 5: 'Very Satisfied'}"
2,linear_scale,recommendation_likelihood,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",How likely are you to recommend this store to a friend or family member?,"{0: 'Not at all likely', 10: 'Extremely likely'}"
3,linear_scale,store_cleanliness,"[1, 2, 3, 4, 5]",How would you rate the cleanliness of the store?,{}
4,linear_scale,staff_friendliness,"[1, 2, 3, 4, 5]",How would you rate the friendliness of our staff?,{}
5,linear_scale,product_variety,"[1, 2, 3, 4, 5]",How would you rate the variety of products available?,{}
6,linear_scale,product_prices,"[1, 2, 3, 4, 5]",How would you rate the prices of our products?,{}
7,multiple_choice,age_range,"['Under 18', '18-24', '25-34', '35-44', '45-54', '55-64', '65 or older']",What is your age range?,nan
8,multiple_choice,gender,"['Male', 'Female', 'Non-binary', 'Prefer not to say']",What is your gender?,nan
9,free_text,additional_comments,nan,Do you have any additional comments or suggestions for improvement?,nan


## Designing AI agents
EDSL comes with methods for designing AI agent personas for language models to use in answering questions.
An `Agent` is created by passing a dictionary of relevant `traits`. It can then be assigned to a survey using the `by()` method when the survey is run (the same as we do with scenarios and models).

We can import existing data to create agents representing audiences of interest, or use EDSL to generate personas:

In [23]:
q_personas = QuestionList(
    question_name="personas",
    question_text="Draft 5 diverse personas for customers of a landscape business in New England capable of answering a feedback survey."
)

If we do not specify a model to use in running the question, the default model GPT 4 preview is used:

In [24]:
personas = q_personas.run().select("personas").to_list()[0]
personas

Job UUID,3e142b8c-3a74-4d47-afc4-a0af18d93468
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/3e142b8c-3a74-4d47-afc4-a0af18d93468
Exceptions Report URL,None
Results UUID,74e2860f-4b01-48cc-b4a4-08de112c5918
Results URL,https://www.expectedparrot.com/content/74e2860f-4b01-48cc-b4a4-08de112c5918


['Retired Couple in Suburbia',
 'Young Urban Professional',
 'Environmentally Conscious Family',
 'Small Business Owner',
 'Historic Home Enthusiast']

Note that the personas can be (much) longer and include key/value pairs for any desired traits; we keep it simple here for demonstration purposes.
Here we pass the personas to a list of agents and have them answer the survey:

In [25]:
from edsl import AgentList, Agent

In [26]:
agents = AgentList(
    Agent(
        traits = {"persona":p},
        instruction = """
        You are answering a customer feedback survey for a landscaping business that you have engaged in the past.
        Your answers are completely confidential.
        """
    )
    for p in personas
)

In [27]:
new_results = new_survey.by(agents).by(models).run()

Job UUID,75e5c169-e906-401c-b421-89461a3afddc
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/75e5c169-e906-401c-b421-89461a3afddc
Exceptions Report URL,None
Results UUID,6d8e728a-97e9-4dd5-ba95-1168535899da
Results URL,https://www.expectedparrot.com/content/6d8e728a-97e9-4dd5-ba95-1168535899da


In [28]:
(
    new_results
    .sort_by("model", "persona")
    .select("model", "persona", "answer.*")
)

,model.model,agent.persona,answer.additional_comments,answer.age_range,answer.product_variety,answer.recommendation_likelihood,answer.product_prices,answer.gender,answer.overall_satisfaction,answer.visit_frequency,answer.staff_friendliness,answer.store_cleanliness
0,claude-3-5-sonnet-20240620,Environmentally Conscious Family,"As an environmentally conscious family, I really appreciated the eco-friendly approach of your landscaping services. However, I have a few suggestions that could further improve your offerings: 1. Native plant options: It would be great to see a wider selection of native plants that support local ecosystems and require less water and maintenance. 2. Organic pest control: I'd love more information on natural pest control methods you use or recommend, avoiding harmful chemicals. 3. Water conservation: Perhaps you could offer installation of rain barrels or smart irrigation systems to help reduce water usage. 4. Composting services: It would be fantastic if you could offer composting services or guidance on how to start a home compost system. 5. Sustainable materials: When possible, use recycled or sustainable materials for hardscaping projects. 6. Educational resources: Provide more information on sustainable gardening practices that we can implement ourselves. 7. Green waste disposal: Ensure all green waste is properly composted or recycled, not sent to landfills. Overall, we've been pleased with your services, but these additions would really align with our family's environmental values and potentially attract more eco-conscious customers.",35-44,4,8,3,Female,4,1,4,4
1,claude-3-5-sonnet-20240620,Historic Home Enthusiast,"As a historic home enthusiast, I really appreciate when landscaping companies understand the unique needs and aesthetics of period properties. I would suggest offering specialized services or packages tailored to historic homes, such as: 1. Heritage plant selections that are appropriate for different architectural eras 2. Expertise in maintaining and restoring historic garden layouts 3. Knowledge of traditional landscaping techniques and materials 4. Ability to work around delicate old structures and mature trees 5. Familiarity with local historic preservation guidelines It would also be great if the staff received some training on the history of landscape design to better serve clients with historic properties. Perhaps partnering with local historical societies or preservation groups could provide additional resources and credibility in this niche. Overall, I've been satisfied with the service, but these additions would really set the company apart for homeowners like myself who are passionate about maintaining the historical integrity of our properties.",55-64,4,8,4,Female,4,1,4,3
2,claude-3-5-sonnet-20240620,Retired Couple in Suburbia,"As a retired couple living in suburbia, we've generally been pleased with the landscaping service. However, we do have a few suggestions that might improve things: 1. We'd appreciate if the crew could be a bit more mindful of noise levels, especially in the early morning hours. As retirees, we tend to enjoy our quiet mornings. 2. It would be helpful to have more flexibility in scheduling. Perhaps offering service time slots in the afternoon could be an option for those of us who aren't on a typical work schedule. 3. We've noticed that some of the younger plants haven't been thriving as well as we'd hoped. A bit more guidance on plant care or perhaps a follow-up visit to check on new plantings would be appreciated. 4. While we understand the importance of chemical treatments for lawn health, we'd be interested in exploring more eco-friendly options if available. 5. Lastly, it might be nice to offer some simple landscaping workshops for customers. As retirees, we have more time to tend to our garden and would love to learn more about maintaining it between your visits. Overall, we're satisfied with the service, but these small improvements could really enh

## Posting to the Coop
[Coop](https://www.expectedparrot.com/content/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we demonstrate how to post this notebook:

In [30]:
from edsl import Notebook

nb = Notebook(path = "google_form_to_edsl.ipynb")

if refresh := False:
    nb.push(
        description = "Example code for using EDSL to convert a non-EDSL survey into EDSL", 
        alias = "google-form-to-edsl-notebook",
        visibility = "public"
    )
else:
    nb.patch('815c1988-8656-4644-8ab8-70ce36b8d62d', value = nb)